In [1]:
from flask import Flask, redirect, url_for, request, render_template, flash
app = Flask(__name__)
import flask
import sqlite3
from flask_mysqldb import MySQL
from flask_login import user_loaded_from_header
from flask_login import LoginManager, login_user
from forms import LoginForm




app.secret_key = "ziyadkhan051"



##########################################################
######              MYSQL CONNNECTION                #####
##########################################################

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''
app.config['MYSQL_DB'] = 'posts'

mysql = MySQL(app)




@app.route('/success/<name>')
def success(name):
   return 'welcome %s' % name

##########################################################
######              DB CONNECTION                    #####
##########################################################
def get_db_connection():
   conn = sqlite3.connect('database.db')
   conn.row_factory = sqlite3.Row
   return conn


@app.route('/apps')
def apps():
   conn = get_db_connection()
   posts = conn.execute("select * from posts").fetchall()
   return render_template("posts.html",posts=posts)
##########################################################


@app.route('/login',methods = ['POST', 'GET'])
def login():
   if request.method == "POST":
      user = request.form['nm']
      data = {
      "user":user,
      }
      return render_template("success.html",data=data)
   else:
      data = {}
      return render_template("login.html",data=data)


@app.route("/get-data",methods = ['GET'])
def get_data():
   info = flask.request.values.get('first_name')
   return render_template("get_data.html",data=info)



@app.route("/get-data-mysql",methods = ['GET','POST'])
def get_data_mysql():
   if request.method == "POST":
      firstName = flask.request.values.get('first_name')
      lastName = flask.request.values.get('last_name')
      cursor = mysql.connection.cursor()
      cursor.execute("INSERT INTO MyUsers(firstName, lastName) VALUES (%s, %s)", (firstName, lastName))
      mysql.connection.commit()
      cursor.close()
      flash("Successfully registered!")
      return render_template("get_data_mysql_success.html")
   else:
      return render_template("get_data_mysql.html")


@app.route("/test")
def login_user_test():
   form = LoginForm(request.form)
   if request.method == "POST" and form.validate():
      
      return "SUCCESS!"
   else:
      return render_template("login_test.html",form=form)
   


login.html

<html>
   <body>
      <form action = "http://localhost:5000/login" method = "post">
         <p>Enter Name:</p>
         <p><input type = "text" name = "nm" /></p>
         <p><input type = "submit" value = "submit" /></p>
      </form>   
   </body>
</html>

success.html


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SUccess login</title>

</head>
<body>  
    

{% if data.user == "ulvi" %}
Welcome, {{data.user}}, that's corrent!
{% elif data.user == "asd" %}
Welcome, asd, he bu asd dir
{% else %}
Welcome others.... {{data.user}}
{% endif %}

<br>
{% for i in data %}
{{i}}
{% endfor %}    
</body>
</html>

schema.sql


DROP TABLE IF EXISTS posts;

CREATE TABLE posts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    created TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    title TEXT NOT NULL,
    content TEXT NOT NULL
);

init_db.py



import sqlite3

connection = sqlite3.connect("database.db")

with open("schema.sql") as f:
    connection.executescript(f.read())

cur = connection.cursor()

cur.execute("INSERT INTO posts (title, content) VALUES (?, ?)",
            ('First Post', 'Content for the first post')
            )

cur.execute("INSERT INTO posts (title, content) VALUES (?, ?)",
            ('Second Post', 'Content for the second post')
            )

connection.commit()
connection.close()

forms.py



from wtforms import Form, BooleanField, StringField, PasswordField, validators

class LoginForm(Form):
    username = StringField("Username", [validators.Length(min=4,max=40)])
    password = PasswordField("Password", [validators.Length(min=7, max=200)])
    confirm = PasswordField("Confirm")

In [ ]:
{% with messages = get_flashed_messages() %}
    {% if messages %}
      <ul class="alert alert-success">
      {% for message in messages %}
        <center> {{ message }}</center>
      {% endfor %}
      </ul>
    {% endif %}
  {% endwith %}